# End of week 1 exercise solution

## Technical Question Explainer Tool

This notebook demonstrates a tool that takes a technical question and responds with an explanation using both OpenAI (gpt-4o-mini) and Ollama (llama3.2) models.

---

## Step 1: Import Required Libraries

We need the following libraries:
- `os`, `dotenv` for environment variables
- `openai` for OpenAI API
- `ollama` for local LLM

> If not installed, run `pip install openai python-dotenv ollama`

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from openai import OpenAI
import ollama

In [ ]:
load_dotenv(override=True)

youtube_api_key = os.getenv("YOUTUBE_API_KEY")
openrouter_api_key = os.getenv("OPEN_ROUTER_API_KEY")

if not youtube_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not youtube_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")

if not openrouter_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not openrouter_api_key.startswith("sk-or-v1-"):
    print("An Open Router API key was found, but it doesn't start sk-or-v1-")
else:
    print("An Open Router API key found and looks good so far!")

In [ ]:
# Model constants
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3:latest'

---

## Step 2: Enter your technical questions as a list

Type your technical questions below. You can change the question and re-run the notebook to get new answers.

In [ ]:
# Enter your technical questions here
questions = [
    "Please explain what this code does and why: yield from {book.get('author') for book in books if book.get('author')}",
    "What is the difference between a list and a tuple in Python?",
    "How does a neural network learn?",
    "What is the purpose of the __init__ method in Python classes?",
    "Explain the concept of recursion with an example."
]
# You can change the questions and re-run the notebook to get new answers.

---

## Step 3: Stream Answers from OpenAI (gpt-4o-mini)

This cell streams your technical questions to OpenAI's gpt-4o-mini model and displays the answers live as they are generated.

You will see each question and its answer appear in real time.

In [ ]:
# Stream OpenAI responses for all questions (gpt-4o-mini)
from IPython.display import display, Markdown, update_display

def stream_openai_response(question):
    if not openrouter_api_key:
        print("No OpenRouter API key found.")
        return
    openai = OpenAI(api_key=openrouter_api_key, base_url="https://openrouter.ai/api/v1")
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[{"role": "user", "content": question}],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)
    return response

# Stream all questions and display answers
for i, q in enumerate(questions):
    print(f"\033[1mQ{i+1}: {q}\033[0m")
    print("OpenAI Streaming Answer:")
    stream_openai_response(q)
    print("\n---\n")

---

## Step 4: Stream Answers from Ollama (llama3:latest)

This cell streams your technical questions to the local Ollama llama3:latest model and displays the answers live as they are generated.

You will see each question and its answer appear in real time.

In [ ]:
# Stream answers from Ollama (llama3:latest)
def stream_ollama_response(question):
    response = ""
    stream = ollama.chat(model=MODEL_LLAMA, messages=[{"role": "user", "content": question}], stream=True)
    for chunk in stream:
        if 'message' in chunk and 'content' in chunk['message']:
            print(chunk['message']['content'], end='', flush=True)
            response += chunk['message']['content']
    print("\n")
    return response

# Stream all questions and display answers from Ollama
for i, q in enumerate(questions):
    print(f"\033[1mQ{i+1}: {q}\033[0m")
    print("Ollama Streaming Answer:")
    stream_ollama_response(q)
    print("\n---\n")